In [9]:
import pennylane as qml
from pennylane import numpy as np

np.set_printoptions(precision=10)

In [10]:
dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def circuit(params):
    qml.RY(params[0], wires=0)
    qml.RX(params[1], wires=1)
    return qml.expval(qml.PauliZ(0) + qml.PauliZ(1))

In [11]:
params = np.array([np.pi/4, np.pi/4])
print(circuit(params))

1.414213562373095


In [13]:
def my_finite_diff_grad(params, h=1.0e-7):
    gradient = np.zeros_like(params)
    
    for i in range(len(params)):
        params[i] += h
        gradient[i] += circuit(params)
        
        params[i] -= 2*h
        gradient[i] -= circuit(params)
        
        gradient[i] /= 2*h 
        
        params[i] += h
        
    return gradient

@qml.qnode(dev, diff_method="finite-diff")
def circuit_finite_diff(params):
    qml.RY(params[0], wires=0)
    qml.RX(params[1], wires=1)
    return qml.expval(qml.PauliZ(0) + qml.PauliZ(1))

In [16]:
params = np.array([np.pi/4, np.pi/4], requires_grad=True)

print(my_finite_diff_grad(params))
print(qml.grad(circuit_finite_diff)(params))

[-0.7071067798 -0.707106782 ]
[-0.7071068153 -0.7071068175]


In [20]:
def my_parameter_shift_grad(params, s=np.pi/3):
    gradient = np.zeros_like(params)
    
    for i in range(len(params)):
        params[i] += s
        gradient[i] += circuit(params)
        
        params[i] -= 2*s
        gradient[i] -= circuit(params)
        
        gradient[i] /= 2*np.sin(s)
        
        params[i] += s
        
    return gradient
   
@qml.qnode(dev, diff_method="parameter-shift")
def circuit_parameter_shift(params):
    qml.RY(params[0], wires=0)
    qml.RX(params[1], wires=1)
    return qml.expval(qml.PauliZ(0) + qml.PauliZ(1))

In [21]:
print(my_parameter_shift_grad(params))
print(qml.grad(circuit_parameter_shift)(params))

[-0.7071067812 -0.7071067812]
[-0.7071067812 -0.7071067812]
